<a href="https://colab.research.google.com/github/ImagingDataCommons/IDC-Examples/blob/master/API/notebooks/How_to_use_IDC_APIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to Use the IDC APIs

## Overview of this notebook
This notebook is designed as a quick introduction to the IDC APIs and how to access them with Python.

Topics covered:
* Overviews of APIs, Swagger, JSON, endpoints
* Use cases for IDC APIs
* Examples of IDC API endpoints

### Overview of APIs
An API or application-programming interface is a software intermediary that allows two applications to talk to each other. In other words, an API is the messenger that delivers your request to the provider that you’re requesting it from and then delivers the response back to you [(Wikipedia)](https://en.wikipedia.org/wiki/Application_programming_interface). Each action that an API can take is called an "endpoint".

Some useful tutorials and quick start guides on APIs are:
* [GDC's Getting Started guide for APIs](https://docs.gdc.cancer.gov/API/Users_Guide/Getting_Started/)
* [API Integration in Python](https://realpython.com/api-integration-in-python/)
* [Python API Tutorial: Getting Started with APIs](https://www.dataquest.io/blog/python-api-tutorial/)

### What is an HTTP Message?

Clients and the IDC API server communicate via HTTP messages. An overview of HTTP messaging can be found [here](https://developer.mozilla.org/en-US/docs/Web/HTTP/Messages).

### What is JSON?

JSON  or JavaScript Object Notation is a lightweight data-interchange format that is easy for humans and machines to work with. More information can be found at [json.org](https://www.json.org/).

### What is SwaggerUI?

[SwaggerUI](https://swagger.io/tools/swagger-ui/) is a user interface that allows users to try out the APIs and view their documentation easily. You can access the IDC API SwaggerUI [here](https://api-dot-idc-dev.appspot.com/v1/swagger).

### What is an endpoint?

An endpoint is the *call* for a specific functionally of an API. For example, `/collections` at the end of the API request URL `https://api-dot-idc.appspot.com/v1/collections` is an endpoint that returns (or GETs) information about the available collections.

###IDC API Documentation

Detailed documentation on the IDC API can be found in the [API](https://learn.canceridc.dev/api/getting-started) section of the [IDC User Guide](https://learn.canceridc.dev/ps://).

### IDC API URL Preamble
We define the IDC API URL preamble as a variable so that we can easily change it.

In [1]:
idc_api_preamble = 'https://api-dot-idc-dev.appspot.com/v1'

### Python library `requests`

In order to use the IDC APIs with Python, the `requests` library needs to be installed and then imported.

In [2]:
# Install requests if needed
# pip install requests

# Import the requests library
import requests

## Use cases for IDC APIs

The IDC APIs can be used for a number of different tasks for interacting with the Google Cloud Platform and BigQuery. It can be used to subset data into cohorts or to access cohorts that have been created using the IDC WebApp. The location of the DICOM objects associated with a cohort can be obtained.

## Example: GET `/about` Endpoint



We are first going to explore the `about` endpoint using the 'GET' request to the API. This API will give you such information about the IDC API as links to the Swagger UI interface and to the IDC User Guide.

In [3]:
# First submit the 'get' request to the API
about_req = requests.get('{}/about'.format(idc_api_preamble))

Now that we have the request response, we are going to check that we didn't receive an error code or if the request was successful. If the request was successful, then the status code will come back as 200 but if something went wrong then the status code may be something 404 or 503. If you have recieved any error codes, you can check out Google's Troubleshooting response errors guide.



In [4]:
# Check that there wasn't an error with the request
if about_req.status_code != 200:
  # Print the error code if something went wrong
  print(about_req.status_code)

Finally, we will print out the information that we have received from the API. This response returns as a dictionary though responses can also be a combination of dictionaries and lists depending on which endpoint is called. This means that you can access different data in the response the same way that you would access dictionaries and lists as demonstarted below.

In [5]:
# Print the full response
print("Full response:\n")
print(about_req.json(), end='\n\n')

# Print the message portion of the response
print("Message:\n")
print(about_req.json()['message'], end='\n\n')

# Print the documentation portion of the response
print("Documentation:\n")
print(about_req.json()['documentation'])

Full response:

{'code': 200, 'documentation': 'SwaggerUI interface available at <https://api-dot-idc-dev.appspot.com/v1/swagger/>. Documentation is available at <https://learn.canceridc.dev/>', 'message': 'Welcome to the NCI IDC API, Version 1'}

Message:

Welcome to the NCI IDC API, Version 1

Documentation:

SwaggerUI interface available at <https://api-dot-idc-dev.appspot.com/v1/swagger/>. Documentation is available at <https://learn.canceridc.dev/>


The requests library makes it easy to use the APIs! Next we will cover a few of the other informational APIs.

## Example: GET `/versions` endpoint

Over time, the set of data hosted by the IDC will change. For the most part, such changes will be due to new data having been added. The totality of IDC hosted data resulting from any such change is represented by a unique IDC data version ID. That is, each time that the set of publicly available data changes, a new IDC version is created that exactly defines the revised data set. 

The IDC data version is intended to enable the reproducibility of research results. For example, consider a patient in the DICOM data model. Over time, new studies might be performed on a patient and become associated with that patient, and the corresponding DICOM instances added to the IDC hosted data. Moreover, additional patients might well be added to the IDC data set over time. This means that the set of subjects defined by some filter set will change over time. Thus, for purposes of reproducibility, we define a cohort in terms of a filter set and an IDC data version.

The `/versions` endpoint returns information about the defined IDC versions. This information includes the data sources (BQ tables) containing the data of each version, as well as the set of programs (sets of collections) belonging to a version. This endpoint returns a more complicated JSON object which has a combination of lists and dictionaries. We will first retrieve the request and then view if there was an error code within the response.

In [6]:
# Retrieve the response from the API endpoint
versions_req = requests.get('{}/versions'.format(idc_api_preamble))

# Check that there wasn't an error with the request
if versions_req.status_code != 200:
  # Print the error code and message if something went wrong
  print(versions_req.json())  # Print the error code if something went wrong

We are going to use the `json` library in order to view the response more easily.

In [7]:
# Install requests if needed
# pip install requests

# install pip json
import json

In [8]:
# Create a variable with the JSON output
versions_json = json.dumps(versions_req.json(), sort_keys=True, indent=4)

# Print the versions JSON text
print(versions_json)

{
    "code": 200,
    "versions": [
        {
            "active": true,
            "data_sources": [
                {
                    "data_type": "Clinical, Biospecimen, and Mutation Data",
                    "name": "isb-cgc.TCGA_bioclin_v0.Biospecimen"
                },
                {
                    "data_type": "Clinical, Biospecimen, and Mutation Data",
                    "name": "isb-cgc.TCGA_bioclin_v0.clinical_v1"
                },
                {
                    "data_type": "Image Data",
                    "name": "idc-dev.metadata.dicom_pivot_wave1"
                }
            ],
            "date_active": "2020-10-06",
            "idc_data_version": "1.0",
            "programs": [
                {
                    "description": null,
                    "name": "The Cancer Genome Atlas",
                    "short_name": "TCGA"
                },
                {
                    "description": null,
                    "name": "Quan

The returned data is a combination of dictionaries and lists. We see  that, as of this writing, there is a single IDC version, "1.0", that was activated on 2020-10-06. Next we will iterate over the JSON object to neatly list the data sources and programs in each version.

In [9]:
# Print out the number of IDC data versions
print('Number of IDC data versions: {}'.format(len(versions_req.json()['versions'])))

#...and for each version, print out a count of the version's programs, and a list of program names.
for version in versions_req.json()['versions']:
  print('version {} has {} programs:'.format(version['idc_data_version'], len(version['programs'])))
  for program in version['programs']:
    print('\tProgram short name {}, full name {}'.format(program['short_name'],program['name']))


Number of IDC data versions: 1
version 1.0 has 5 programs:
	Program short name TCGA, full name The Cancer Genome Atlas
	Program short name QIN, full name Quantitative Imagine Network
	Program short name ISPY, full name I-SPY TRIAL
	Program short name LIDC, full name Lung Image Database Consortium
	Program short name NSCLCR, full name NSCLC-Radiomics


## Example: GET `/collections` endpoint

A *collection* is a set of DICOM data provided by a single source. Collections are further categorized as Original collections or Analysis collections. 

Original collections are comprised primarily of DICOM image data that was obtained from some set of patients. Typically, the patients in an Original collection are related by a common disease.

Analysis collections are comprised of derived DICOM data that was generated by analyzing other (typically Original) collections. Typically such analysis is performed by a different entity than that which provided the original collection(s) on which the analysis is based. Examples of data in analysis collections include segmentations, annotations and further processing of original images. Note that some Original collections include such data, though most of the data in Original collections are original images.
Programs

The programs that we listed above are sets of original collections. The collections in a program are produced by a single source. Some programs provide additional non-imaging data. For example, the TCGA program provides extensive ancillary clinical and genomics data about each of the patients in the program. 

The /collections endpoint returns data about collections in a specified program for some IDC data version. If no collection is specified, it returns data about about the collections in all programs for some IDC data version. If a version is not specified then /collections defaults to the current IDC data version.


We will request the collection data for the TCGA program in IDC data version 1.0.

The program and version are passed as *query parameters* in a *query string*. The requests library accepts a dictionary of query parameters.

In [10]:
query_string = dict(
    program_name = "TCGA",
    idc_data_version = "1.0"
    )

collections_req = requests.get('{}/collections'.format(idc_api_preamble),
                    params=query_string)
# Check that there wasn't an error with the request
if collections_req.status_code != 200:
  # Print the error code and message if something went wrong
  print(collections_req.json())

In [11]:
# Create a variable with the JSON output
collections_json = json.dumps(collections_req.json(), sort_keys=True, indent=4)

# Print the verscollectionsions JSON text
print(collections_json)

{
    "code": 200,
    "programs": [
        {
            "collections": [
                {
                    "active": true,
                    "cancer_type": "Prostate Cancer",
                    "collection_id": "tcga_prad",
                    "collection_type": "Original",
                    "date_updated": "2016-08-29",
                    "description": "<div>\n\t<strong>Note:&nbsp;This collection has special restrictions on its usage. See <a href=\"https://wiki.cancerimagingarchive.net/x/c4hF\" target=\"_blank\">Data Usage Policies and Restrictions</a>.</strong></p>\n<div>\n\t&nbsp;</p>\n<div>\n\t<span>The <a href=\"http://imaging.cancer.gov/\" target=\"_blank\"><u>Cancer Imaging Program (CIP)</u></a></span><span>&thinsp;</span><span> is working directly with primary investigators from institutes participating in TCGA to obtain and load images relating to the genomic, clinical, and pathological data being stored within the <a href=\"http://tcga-data.nci.nih.gov/\" target

Metadata for original collections has been obtained from the [TCIA Data Collections page](https://www.cancerimagingarchive.net/collections/), and for analysis collections from the [TCIA Analysis Results page](https://www.cancerimagingarchive.net/tcia-analysis-results/). Not that the `idc_data_versions` component of each collection is a list because any particular collection will almost certainly be available in multiple IDC data versions.

## Example: POST `/cohorts/preview/manifest` endpoint

The POST `/cohorts/preview/manifest` endpoint returns a manifest of *access_methods* for the objects in cohort cohort_id. Please refer to the in the [API](https://learn.canceridc.dev/api/getting-started) section of the [IDC User Guide](https://learn.canceridc.dev/ps://) for further information on manifests. 

The `/cohorts/preview/manifest` API does not actually create a cohort, but acts as if a cohort were created. Creating a cohort requires authenticating to the API; that process is addessed in a subsequent example describing the `/cohorts/{cohort_id}/manifest` API.

A manifest is a list of *access methods*. Each access method describes how to access the study, series or instance DICOM objects in the cohort. A manifest can optionally include addition metadata per DICOM object. 

A filterSet includes a filter and the IDC data version against which it is applied. Filter is a list of *attribute*,*values* pairs, where *values* is a list of one or more values which must be satisfied by the associated attribute.

In the following, we construct a dict, `cohortSpec`, containing the name and a description for the cohort, as well as a filterSet that selects for subjects in either the TCGA-LUAD or TCGA-KIRC collections, having DICOM data with a CT or MR modality, and are Asian.



In [82]:
filterSet = {
    "idc_data_version": "1.0",
    "filters": {
        "collection_id": [ "TCGA-LUAD", "TCGA-KIRC" ],
        "Modality": ["CT", "MR"],
        "race": ["ASIAN"]}}

cohortSpec = {"name": "testcohort",
              "description": "Test description",
              "filterSet": filterSet}



The query string selects additonal metadata to be return. In addition, the amount of data returned by each call can be limited. When this is done the API can be iteratively called until all data has been received. In this example, we will limit the returned data to 10 rows.

In [83]:
query_string = dict(
    sql = False,
    Collection_IDs = True,
    Patient_IDs = True,
    StudyInstanceUIDs = False,
    SeriesInstanceUIDs = False,
    SOPInstanceUIDs = False,
    Collection_DOIs = False,
    access_method =  'url',
    page_size = 10
)

We are now ready to call the endpoint. Note that /cohorts/preview/manifest is a POST method, so we call requests.post()

In [84]:
response = requests.post('{}/cohorts/preview/manifest'.format(idc_api_preamble),
                    params=query_string, json=cohortSpec)

# Check that there wasn't an error with the request
if response.status_code != 200:
  # Print the error code and message if something went wrong
  print(response.json())

We will prettyprint the results for easier comprehension:

In [85]:
print(json.dumps(response.json(), sort_keys=True, indent=4))

{
    "code": 200,
    "cohort": {
        "description": "Test description",
        "filterSet": {
            "filters": {
                "Modality": [
                    "CT",
                    "MR"
                ],
                "collection_id": [
                    "tcga_luad",
                    "tcga_kirc"
                ],
                "race": [
                    "ASIAN"
                ]
            },
            "idc_data_version": "1.0"
        },
        "name": "testcohort",
        "sql": ""
    },
    "manifest": {
        "json_manifest": [
            {
                "PatientID": "TCGA-CJ-4899",
                "collection_id": "tcga_kirc",
                "url": "gs://idc-tcia-tcga-kirc/dicom/1.3.6.1.4.1.14519.5.2.1.1706.4004.222390132567115084999595655241/1.3.6.1.4.1.14519.5.2.1.1706.4004.209568983417342975065810158572/1.3.6.1.4.1.14519.5.2.1.1706.4004.104452021937501750559914839064.dcm#1592634614253420"
            },
            {
              

The returned data includes the cohort name, description and filterset which we passed as parameters.  

We can see that there are 1581 total rows, but only 10 were returned.

We could also have requested that the BigQuery SQL that produced these results be returned.


### Paged results

It can be seen that the above results includes a non-null `next_page` token. This token can be passed as a parameter in a subsequent invocation of /cohorts/preview/manifest to obtain additional data.

When we pass a next_page value, all other parameters except page_size are ignored.


In [87]:
query_string = dict(
    page_size = 10,
    next_page = response.json()['next_page']
)
response = requests.post('{}/cohorts/preview/manifest'.format(idc_api_preamble),
                    params=query_string, json=cohortSpec)

# Check that there wasn't an error with the request
if response.status_code != 200:
  # Print the error code and message if something went wrong
  print(response.json())

print(json.dumps(response.json(), sort_keys=True, indent=4))



{
    "code": 200,
    "cohort": {},
    "manifest": {
        "json_manifest": [
            {
                "PatientID": "TCGA-CJ-4899",
                "collection_id": "tcga_kirc",
                "url": "gs://idc-tcia-tcga-kirc/dicom/1.3.6.1.4.1.14519.5.2.1.1706.4004.222390132567115084999595655241/1.3.6.1.4.1.14519.5.2.1.1706.4004.209568983417342975065810158572/1.3.6.1.4.1.14519.5.2.1.1706.4004.214212115005456573927153866312.dcm#1592634614427958"
            },
            {
                "PatientID": "TCGA-CJ-4899",
                "collection_id": "tcga_kirc",
                "url": "gs://idc-tcia-tcga-kirc/dicom/1.3.6.1.4.1.14519.5.2.1.1706.4004.222390132567115084999595655241/1.3.6.1.4.1.14519.5.2.1.1706.4004.209568983417342975065810158572/1.3.6.1.4.1.14519.5.2.1.1706.4004.216153614447829865047126869441.dcm#1592634614446319"
            },
            {
                "PatientID": "TCGA-CJ-4899",
                "collection_id": "tcga_kirc",
                "url": "gs://id

## Example: POST `/cohort/{cohort_id}/manifest` Endpoint

This section will focus on the POST `/cohorts/{cohort_id}/manifest` endpoint. Unlike the /cohort/preview/manifest endpoint, the `/cohorts/{cohort_id}/manifest` returns a manifest against a cohort that was previously defined via the IDC POST /cohorts API or through the IDC webapp.

In order to be able to save a cohort definition in the IDC webapp via the API, the user must authenticate their identity with the web app. his section will first step through the authentication/authorization process when working with the IDC API.


### Notes on Authorization and Credentials
The following steps are required to use an API that Requires Authorization:

1. Create a Credential File on your local machine by using the [idc_auth.py](https://github.com/ImagingDataCommons/IDC-API/tree/master/scripts/idc_auth.py) script from the [IDC-API Repository](https://github.com/ImagingDataCommons/IDC-API.git)
  * This script can be run from the command line or from within Python but should be run on your local machine.
2. Find the location of the Credential File on your local machine
  * By default, the script will save the file in the user's folder with the file name: ".idc_credentials"
3. Load the Credential file into the cloud environment you are using.

In [88]:
# If you skipped earlier sections, you will need these two packages to run the
# code below
# Install requests if needed
#pip install requests

# Install pip json
#import json

# Import the requests library
#import requests

In [89]:
# import os
import os
# Import files helper for Colab
from google.colab import files

In [90]:
# First delete any existing .idc_credentials. Otherwise file.upload() will name the new file .idc_credentials (1)
try:
  os.remove('./.idc_credentials')
except:
  print('.idc_credentials not found')

# Upload your credentials to the cloud environment
uploaded = files.upload()

Saving .idc_credentials to .idc_credentials


Now that we have the Credentials file created and uploaded to the cloud environment, we can open the file to create the header information need for the API to verify that you have Authorization.

In [91]:
# Open the credentials file
token = open(".idc_credentials", "r")
# Create a json object from teh credential file
token = json.loads(token.read())
# Get Credentials from the token
creds = token['token_response']['id_token']
# Create a json object for requests header
head = {'Authorization': 'Bearer ' + creds}

**Note:** the credentials file will expire after 1 hour and a new one will need to be generated. If a new file is not generated with the idc_auth script, you can delete the original file and try running the script again.

We can now proceed to create a cohort. The POST `/cohorts` endpoint creates a cohort. We will use the same cohort definition as previously.

In [93]:
filterSet = {
  "idc_data_version": "1.0",
  "filters": {
      "collection_id": [ "TCGA-LUAD", "TCGA-KIRC" ],
      "Modality": ["CT", "MR"],
      "race": ["ASIAN"]}}
  
cohortSpec = {"name": "testcohort",
              "description": "Test description",
              "filterSet": filterSet}

response = requests.post('{}/cohorts/'.format(idc_api_preamble),
                      json=cohortSpec, headers = head)

# Check that there wasn't an error with the request
if response.status_code != 200:
    # Print the error code and message if something went wrong
    print(response.json())

print(json.dumps(response.json(), sort_keys=True, indent=1))

cohort_id = response.json()['cohort_properties']['cohort_id']

{
 "code": 200,
 "cohort_properties": {
  "cohort_id": 94,
  "description": "Test description",
  "filterSet": {
   "filters": {
    "Modality": [
     "CT",
     "MR"
    ],
    "collection_id": [
     "TCGA-LUAD",
     "TCGA-KIRC"
    ],
    "race": [
     "ASIAN"
    ]
   },
   "idc_data_version": "1.0"
  },
  "name": "testcohort"
 }
}


Note that the response includes the cohort_id of the newly created cohort. We will use this ID when querying for the cohort's manifest. Note also that the response repeats the filterSet and other cohort metadata.

This time we will return doi access methods.

In [98]:
query_string = dict(
    access_method =  'doi',
    Collection_DOIs = True,
    page_size = 10
)

response = requests.get('{}/cohorts/{}/manifest'.format(idc_api_preamble, cohort_id),
                      params=query_string, headers = head)

# Check that there wasn't an error with the request
if response.status_code != 200:
    # Print the error code and message if something went wrong
    print(response.json())

print(json.dumps(response.json(), sort_keys=True, indent=4))


{
    "code": 200,
    "cohort": {
        "cohort_id": 94,
        "description": "Test description",
        "filterSet": {
            "filters": {
                "Modality": [
                    "CT",
                    "MR"
                ],
                "collection_id": [
                    "TCGA-LUAD",
                    "TCGA-KIRC"
                ],
                "race": [
                    "ASIAN"
                ]
            },
            "idc_data_version": "1.0"
        },
        "name": "testcohort",
        "sql": ""
    },
    "manifest": {
        "json_manifest": [
            {
                "doi": "dg.4DFC/000c8565-76f2-4bc8-9a34-33dd3d3924b3",
                "source_DOI": "10.7937/K9/TCIA.2016.V6PBVTDR"
            },
            {
                "doi": "dg.4DFC/002b13fe-8f12-415a-a5e8-401eedba2909",
                "source_DOI": "10.7937/K9/TCIA.2016.V6PBVTDR"
            },
            {
                "doi": "dg.4DFC/00606631-64e6-4d1c-95dd-

As with POST `/cohorts/preview/manifest`, the returned next_page token can be used in subsequent calls to obtain additional data.

Do not confuse the `source_DOI` returned value with the `doi` returned value. A `source_DOI` is the DOI of a TCIA page that describes the collection. A `source_DOI` can be resolved at https://doi.org.

A `doi` can be resolved to a GA4GH DrsObject. Please refer to the [API](https://learn.canceridc.dev/api/getting-started) section of the [IDC User Guide](https://learn.canceridc.dev/ps://) for further information on DRSObjects and access methods.

##Example: GET /dicomMetadata

For completeness, this last section reviews the GET /dicomMetadata endpoint. This endpoint returns a fixed selection of DICOM metadata for all instances in IDC DICOM data. It is intended for use by other CRDC resources that might need such information for the purpose of aggregating imaging data with other data types.

Because it returns metadata on all DICOM instances, paging must be used to obtain the complete set of results.

In [99]:
query_string = dict(
    page_size = 10
)

response = requests.get('{}/dicomMetadata'.format(idc_api_preamble, cohort_id),
                      params=query_string, headers = head)

# Check that there wasn't an error with the request
if response.status_code != 200:
    # Print the error code and message if something went wrong
    print(response.json())

print(json.dumps(response.json(), sort_keys=True, indent=4))

{
    "code": 200,
    "next_page": "gAAAAABf4Te5KwsExgBRPjm0CF9leLi6R7o85yhYKfx3AEaEHyAmnf5HDa8lUGIizAsDwB_4vIIhkOqtGGnZWeIzBTcqCvPOEWrgFTF_3J4Ma9lBDj2XvLiY9YjQ4306K02de3sx0Qua9d7SGvNuKSZXqrQv8GH90qyaNcDbq3gs5QPnbnXRVXact0T22VvME_fFrgm6rNu8eWUaxfwiMY884hCr-0KxHuuLXMIC-TgGeyi5oGy97S8QIPDu8aHhh0ZFyi2qo0IqHKiOWmT4adzIMp4HSevwaYtnYPh6mGAH1AGUhnfGlx9me1e-pgm4f1LY8qaYDV8e",
    "query_results": {
        "json": [
            {
                "AdditionalPatientHistory": "rectal cancer 1.07 creatinine",
                "Allergies": [
                    "Facility APCFG"
                ],
                "BodyPartExamined": "RECTUM",
                "EthnicGroup": null,
                "ImageType": [
                    "ORIGINAL",
                    "PRIMARY",
                    "M",
                    "DIS2D"
                ],
                "LastMenstrualDate": null,
                "MedicalAlerts": [],
                "Modality": "MR",
                "Occupation": null,
          